In [89]:
import pandas as pd
import numpy as np
import torch
from collections import Counter
import random
import scipy.stats
from sklearn.preprocessing import MinMaxScaler
import joblib

In [90]:
"""labels:
0 - other
1 - head non-standard plane
2 - head standard plane
3 - femur non-standard plane
4 - femur standard plane
5 - femur non standard plane
6 - femur standard plane"""

'labels:\n0 - other\n1 - head non-standard plane\n2 - head standard plane\n3 - femur non-standard plane\n4 - femur standard plane\n5 - femur non standard plane\n6 - femur standard plane'

In [91]:
train = pd.read_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_scaled_ps/biometry_train_scaled_size.csv')
val = pd.read_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_scaled_ps/biometry_val_scaled_size.csv')
test = pd.read_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_scaled_ps/biometry_test_scaled_size.csv')

In [92]:
train_org = pd.read_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_train.csv')
test_org = pd.read_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_test.csv')

val_org = pd.read_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_val.csv')

In [93]:
train['days'] = train_org['days']
test['days'] = test_org['days']
val['days'] = val_org['days']

In [94]:
train_head = train[train['Class'] == 2]
train_abdomen = train[train['Class'] == 4]
train_femur = train[train['Class'] == 6]

val_head = val[val['Class'] == 2]
val_abdomen = val[val['Class'] == 4]
val_femur = val[val['Class'] == 6]

test_head = test[test['Class'] == 2]
test_abdomen = test[test['Class'] == 4]
test_femur = test[test['Class'] == 6]

In [95]:
classes = train_abdomen['video'].str.split('_', expand=True)[1]

In [96]:
classes.value_counts()

2    2933
3      24
Name: 1, dtype: int64

In [97]:
train_abdomen_filtered = train_abdomen[classes == '2']
test_abdomen_filtered = test_abdomen[test_abdomen['video'].str.split('_', expand=True)[1] == '2']
val_abdomen_filtered = val_abdomen[val_abdomen['video'].str.split('_', expand=True)[1] == '2']

In [98]:
train_head_filtered = train_head[train_head['video'].str.split('_', expand=True)[1] == '1']
test_head_filtered = test_head[test_head['video'].str.split('_', expand=True)[1] == '1']
val_head_filtered = val_head[val_head['video'].str.split('_', expand=True)[1] == '1']

In [99]:
train_femur_filtered = train_femur[train_femur['video'].str.split('_', expand=True)[1] == '3']
test_femur_filtered = test_femur[test_femur['video'].str.split('_', expand=True)[1] == '3']
val_femur_filtered = val_femur[val_femur['video'].str.split('_', expand=True)[1] == '3']

In [100]:
train_abdomen_filtered

,index,Class,video,measures,ps,frames_n,measure_scaled,days,frame_loc,height,width,height_org,width_org,measure_normalized
226,6_2_35,4,6_2,16.47,0.120130,51,0.598042,147,0.686275,86.49360,115.32480,720,960,0.457089
227,6_2_36,4,6_2,16.47,0.120130,51,0.598042,147,0.705882,86.49360,115.32480,720,960,0.457089
228,6_2_37,4,6_2,16.47,0.120130,51,0.598042,147,0.725490,86.49360,115.32480,720,960,0.457089
229,6_2_38,4,6_2,16.47,0.120130,51,0.598042,147,0.745098,86.49360,115.32480,720,960,0.457089
230,6_2_39,4,6_2,16.47,0.120130,51,0.598042,147,0.764706,86.49360,115.32480,720,960,0.457089
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134553,688_2_31,4,688_2,24.38,0.139701,36,0.774758,192,0.861111,100.58472,134.11296,720,960,0.705753
134554,688_2_32,4,688_2,24.38,0.139701,36,0.774758,192,0.888889,100.58472,134.11296,720,960,0.705753
134555,688_2_33,4,688_2,24.38,0.139701,36,0.774758,192,0.916667,100.58472,134.11296,720,960,0.705753
135147,696_2_91,4,696_2,21.41,0.199092,103,0.458407,183,0.883495,143.34624,191.12832,720,960,0.612386


In [101]:
train_abdomen_filtered['measure_scaled'] = train_abdomen_filtered['measures']/train_abdomen_filtered['ps']
test_abdomen_filtered['measure_scaled'] = test_abdomen_filtered['measures']/test_abdomen_filtered['ps']
val_abdomen_filtered['measure_scaled'] = val_abdomen_filtered['measures']/val_abdomen_filtered['ps']

/data/kpusteln/my-envs/swin/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/data/kpusteln/my-envs/swin/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/data/kpusteln/my-envs/swin/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_index

In [102]:
scaler = MinMaxScaler()
measure_scaled = train_abdomen_filtered['measure_scaled']
scaler.fit(measure_scaled.values.reshape(-1,1))

MinMaxScaler()

In [103]:
train_abdomen_filtered['measure_scaled'] = scaler.transform(train_abdomen_filtered['measure_scaled'].values.reshape(-1,1))
test_abdomen_filtered['measure_scaled'] = scaler.transform(test_abdomen_filtered['measure_scaled'].values.reshape(-1,1))
val_abdomen_filtered['measure_scaled'] = scaler.transform(val_abdomen_filtered['measure_scaled'].values.reshape(-1,1))

/data/kpusteln/my-envs/swin/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/data/kpusteln/my-envs/swin/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/data/kpusteln/my-envs/swin/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_index

In [104]:
scaler.inverse_transform(train_abdomen_filtered['measure_scaled'].values.reshape(-1,1))

array([[137.1014734 ],
       [137.1014734 ],
       [137.1014734 ],
       ...,
       [174.51557254],
       [107.53822353],
       [107.53822353]])

In [105]:
joblib.dump(scaler, 'abdomen_scaler')

['abdomen_scaler']

In [106]:
train_femur_filtered['measure_scaled'] = train_femur_filtered['measures']/train_femur_filtered['ps']
test_femur_filtered['measure_scaled'] = test_femur_filtered['measures']/test_femur_filtered['ps']
val_femur_filtered['measure_scaled'] = val_femur_filtered['measures']/val_femur_filtered['ps']

/data/kpusteln/my-envs/swin/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/data/kpusteln/my-envs/swin/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [107]:
scaler = MinMaxScaler()
measure_scaled = train_femur_filtered['measure_scaled']
scaler.fit(measure_scaled.values.reshape(-1,1))

MinMaxScaler()

In [108]:
train_femur_filtered['measure_scaled'] = scaler.transform(train_femur_filtered['measure_scaled'].values.reshape(-1,1))
test_femur_filtered['measure_scaled'] = scaler.transform(test_femur_filtered['measure_scaled'].values.reshape(-1,1))
val_femur_filtered['measure_scaled'] = scaler.transform(val_femur_filtered['measure_scaled'].values.reshape(-1,1))

/data/kpusteln/my-envs/swin/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/data/kpusteln/my-envs/swin/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [109]:
scaler.inverse_transform(train_femur_filtered['measure_scaled'].values.reshape(-1,1))

array([[47.37117519],
       [47.37117519],
       [47.37117519],
       ...,
       [28.81139571],
       [28.81139571],
       [28.81139571]])

In [110]:
joblib.dump(scaler, 'femur_scaler')

['femur_scaler']

In [111]:
train_head_filtered['measure_scaled'] = train_head_filtered['measures']/train_head_filtered['ps']
test_head_filtered['measure_scaled'] = test_head_filtered['measures']/test_head_filtered['ps']
val_head_filtered['measure_scaled'] = val_head_filtered['measures']/val_head_filtered['ps']

/data/kpusteln/my-envs/swin/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/data/kpusteln/my-envs/swin/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/data/kpusteln/my-envs/swin/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_index

In [112]:
scaler = MinMaxScaler()
measure_scaled = train_head_filtered['measure_scaled']
scaler.fit(measure_scaled.values.reshape(-1,1))

MinMaxScaler()

In [113]:
train_head_filtered['measure_scaled'] = scaler.transform(train_head_filtered['measure_scaled'].values.reshape(-1,1))
test_head_filtered['measure_scaled'] = scaler.transform(test_head_filtered['measure_scaled'].values.reshape(-1,1))
val_head_filtered['measure_scaled'] = scaler.transform(val_head_filtered['measure_scaled'].values.reshape(-1,1))

/data/kpusteln/my-envs/swin/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/data/kpusteln/my-envs/swin/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/data/kpusteln/my-envs/swin/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_index

In [114]:
scaler.inverse_transform(train_head_filtered['measure_scaled'].values.reshape(-1,1))

array([[149.17173063],
       [149.17173063],
       [149.17173063],
       ...,
       [104.99098643],
       [104.99098643],
       [ 90.37054159]])

In [115]:
joblib.dump(scaler, 'head_scaler')

['head_scaler']

In [116]:
train_abdomen_filtered.to_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_scaled_ps/abdomen/biometry_train_scaled_size_abdomen.csv', index=False)
val_abdomen_filtered.to_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_scaled_ps/abdomen/biometry_val_scaled_size_abdomen.csv', index=False)
test_abdomen_filtered.to_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_scaled_ps/abdomen/biometry_test_scaled_size_abdomen.csv', index=False)

In [117]:
train_femur_filtered.to_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_scaled_ps/femur/biometry_train_scaled_size_femur.csv', index=False)
test_femur_filtered.to_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_scaled_ps/femur/biometry_test_scaled_size_femur.csv', index=False)
val_femur_filtered.to_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_scaled_ps/femur/biometry_val_scaled_size_femur.csv', index=False)



In [118]:
train_head_filtered.to_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_scaled_ps/head/biometry_train_scaled_size_head.csv', index=False)
test_head_filtered.to_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_scaled_ps/head/biometry_test_scaled_size_head.csv', index=False)
val_head_filtered.to_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_scaled_ps/head/biometry_val_scaled_size_head.csv', index=False)

In [122]:
train_all = pd.concat([train_abdomen_filtered, train_femur_filtered, train_head_filtered])
test_all = pd.concat([test_abdomen_filtered, test_femur_filtered, test_head_filtered])
val_all = pd.concat([val_abdomen_filtered, val_femur_filtered, val_head_filtered])

In [125]:
train_all['measure_scaled'] = train_all['measures']/train_all['ps']
test_all['measure_scaled'] = test_all['measures']/test_all['ps']
val_all['measure_scaled'] = val_all['measures']/val_all['ps']

In [126]:
train_all

,index,Class,video,measures,ps,frames_n,measure_scaled,days,frame_loc,height,width,height_org,width_org,measure_normalized
226,6_2_35,4,6_2,16.47,0.120130,51,137.101473,147,0.686275,86.493600,115.324800,720,960,0.457089
227,6_2_36,4,6_2,16.47,0.120130,51,137.101473,147,0.705882,86.493600,115.324800,720,960,0.457089
228,6_2_37,4,6_2,16.47,0.120130,51,137.101473,147,0.725490,86.493600,115.324800,720,960,0.457089
229,6_2_38,4,6_2,16.47,0.120130,51,137.101473,147,0.745098,86.493600,115.324800,720,960,0.457089
230,6_2_39,4,6_2,16.47,0.120130,51,137.101473,147,0.764706,86.493600,115.324800,720,960,0.457089
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21422,94_1_182,2,94_1,26.15,0.249069,185,104.990986,167,0.983784,185.058267,242.842275,743,975,0.761396
21423,94_1_183,2,94_1,26.15,0.249069,185,104.990986,167,0.989189,185.058267,242.842275,743,975,0.761396
21424,94_1_184,2,94_1,26.15,0.249069,185,104.990986,167,0.994595,185.058267,242.842275,743,975,0.761396
21425,94_1_185,2,94_1,26.15,0.249069,185,104.990986,167,1.000000,185.058267,242.842275,743,975,0.761396


In [127]:
scaler = MinMaxScaler()
measure_scaled = train_all['measure_scaled']
scaler.fit(measure_scaled.values.reshape(-1,1))

MinMaxScaler()

In [128]:
train_all['measure_scaled'] = scaler.transform(train_all['measure_scaled'].values.reshape(-1,1))
test_all['measure_scaled'] = scaler.transform(test_all['measure_scaled'].values.reshape(-1,1))
val_all['measure_scaled'] = scaler.transform(val_all['measure_scaled'].values.reshape(-1,1))

In [133]:
train_all.to_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_scaled_ps/all/biometry_train_scaled_size_all.csv', index=False)
test_all.to_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_scaled_ps/all/biometry_test_scaled_size_all.csv', index=False)
val_all.to_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_scaled_ps/all/biometry_val_scaled_size_all.csv', index=False)